In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'


# 2) Se procede a crear un dataset simulado
import pandas as pd
from datetime import datetime, timedelta
import random


n = 1000
start = datetime.now()
rows = []
for i in range(n):
	ts = start + timedelta(seconds=random.randint(0, 3600))
	user = f'user_{random.randint(1,50)}'
	clicks = random.randint(1,5)
	rows.append({'Timestamp': ts.strftime('%Y-%m-%d %H:%M:%S'), 'User_ID': user, 'Clicks': clicks})


df_sim = pd.DataFrame(rows)
df_sim.to_csv('clickstream.csv', index=False)
print('CSV simulado guardado: clickstream.csv')

# 3) Se inicia Spark (PySpark) dentro de Colab
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window, sum as spark_sum, to_timestamp


spark = SparkSession.builder \
.appName('ClickstreamAnalysis') \
.config('spark.ui.port', '4041') \
.getOrCreate()


# 4) Cargar CSV (DataFrame) de Spark
sdf = spark.read.option('header', True).csv('clickstream.csv')
# Convertir Timestamp a tipo timestamp y Clicks a entero
sdf = sdf.withColumn('ts', to_timestamp(col('Timestamp'), 'yyyy-MM-dd HH:mm:ss')) \
.withColumn('Clicks', col('Clicks').cast('integer'))
sdf.printSchema()
sdf.show(5, truncate=False)

# 5) Simulación de procesamiento en ventanas de 1 minuto
# Agrupar por ventana (1 minuto y User_ID) y luego se Clicks
result = sdf.groupBy(window(col('ts'), '1 minute'), col('User_ID')) \
.agg(spark_sum('Clicks').alias('clicks_per_min')) \
.orderBy('window')


# Se muestran algunas filas
result.show(10, truncate=False)


# Convertir a pandas con la misión de graficar (pequeño resultado)
pandas_res = result.toPandas()
# Aplanar la ventana (start time)
pandas_res['window_start'] = pandas_res['window'].apply(lambda w: w.start)


# Implementar un resumen por usuario total (sobre todo el periodo simulado)
user_summary = pandas_res.groupby('User_ID')['clicks_per_min'].sum().reset_index()
user_summary = user_summary.sort_values('clicks_per_min', ascending=False)
user_summary.head(10)

# 6) Visualizazión (matplotlib)
import matplotlib.pyplot as plt


# Gráfica con el top 10 de usuarios por clics
top10 = user_summary.head(10)
plt.figure(figsize=(10,6))
plt.bar(top10['User_ID'], top10['clicks_per_min'])
plt.xlabel('User_ID')
plt.ylabel('Total Clicks (ventanas de 1 min)')
plt.title('Top 10 usuarios por clics')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('top10_clicks.png')
plt.show()

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


CSV simulado guardado: clickstream.csv
